In [5]:
import requests
import json
from config import *
import pandas as pd
# Define my API Key, My Endpoint, and My Header
ENDPOINT = 'https://api.yelp.com/v3/businesses/search'.format(client)
headers = {'Authorization': 'bearer %s' % api_key}

Get US zip codes from publicly available Census Data

In [6]:
census_pd = pd.read_csv("../Resources/Census_Data.csv")
census_pd.head()

,Zipcode,Address,Population,Median Age,Household Income,Per Capita Income
0,15081,"South Heights, PA 15081, USA",342,50.2,31500.0,22177
1,20615,"Broomes Island, MD 20615, USA",424,43.4,114375.0,43920
2,50201,"Nevada, IA 50201, USA",8139,40.4,56619.0,28908
3,84020,"Draper, UT 84020, USA",42751,30.4,89922.0,33164
4,39097,"Louise, MS 39097, USA",495,58.0,26838.0,17399


Get only those zip codes with population > 1000

In [7]:
above_1000 = census_pd[census_pd['Population'] >= 1000]
print(f'there are {above_1000.shape[0]} such zip codes in the United States')
df = above_1000.sample(n = 500)  # random sample of 500 zip codes
zip_list = df['Zipcode'].tolist() # list of zip codes
# zip_list

there are 527 such zip codes in the United States


In [8]:
from pandas import json_normalize 
#Food Type data dataframes
mexican = pd.DataFrame(columns = ['name', 'rating', 'review_count', 'weighted_rating'])
italian = pd.DataFrame(columns = ['name', 'rating', 'review_count', 'weighted_rating'])

df_list = []

cuisines = ['Mexican', 'Italian']
for choice in cuisines:
    for zip in zip_list:
        url='https://api.yelp.com/v3/businesses/search'
        params = {'term':choice,'location':zip}
        req=requests.get(url, params=params, headers=headers)
        response = json.loads(req.text)
        df = json_normalize(response, 'businesses')
        try:
            data = df[['name', 'rating', 'review_count']]
            # data['weighted_rating'] = (data['review_count']/data['rating'])
            data['cuisine'] = choice
            df_list.append(data)
        except:
            pass
            # print('columns for name rating and review count not in data')
    

<ipython-input-8-b1516868dd71>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cuisine'] = choice


In [9]:
total_data = pd.concat(df_list)
total_data.head() 
# df_list[:10]

,name,rating,review_count,cuisine
0,El Tio,3.5,51,Mexican
1,Tipicos Gloria Restaurant,4.0,230,Mexican
2,Taco Zocalo,4.5,337,Mexican
3,Casa Tequila Bar & Grill - Sterling,4.5,99,Mexican
4,Locals Tacos and Tequila,4.5,45,Mexican


In [10]:
total_data.shape
total_data['weighted_rating'] = (total_data['review_count']/total_data['rating'])


In [11]:
total_data.head()

,name,rating,review_count,cuisine,weighted_rating
0,El Tio,3.5,51,Mexican,14.571429
1,Tipicos Gloria Restaurant,4.0,230,Mexican,57.500000
2,Taco Zocalo,4.5,337,Mexican,74.888889
3,Casa Tequila Bar & Grill - Sterling,4.5,99,Mexican,22.000000
4,Locals Tacos and Tequila,4.5,45,Mexican,10.000000


In [12]:
mexican = total_data[total_data['cuisine'] == 'Mexican']
italian = total_data[total_data['cuisine'] == 'Italian']

In [13]:
round(italian.weighted_rating.mean(),2)

39.27

In [14]:
mexican.weighted_rating.mean()

31.018777825497565

In [17]:
url='https://api.yelp.com/v3/businesses/search'
# In the dictionary, term can take values like food, cafes or businesses like McDonalds
params = {'term':'Italian','location':'44113'}
# params = {'term':'seafood','zip_code':"44113"}

In [50]:
# Making a get request to the API
req=requests.get(url, params=params, headers=headers)
# proceed only if the status code is 200
print('The status code is {}'.format(req.status_code))

The status code is 200


In [52]:
# printing the text from the response 
response = json.loads(req.text)
from pandas import json_normalize 
df = json_normalize(response, 'businesses')
data = df[['name', 'rating', 'review_count']]
data['weighted_rating'] = (data['review_count']/data['rating'])
data.head()

<ipython-input-52-2a567563437f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['weighted_rating'] = (data['review_count']/data['rating'])


,name,rating,review_count,weighted_rating
0,Luca Italian Cuisine,4.5,294,65.333333
1,Bruno's Ristorante,4.0,204,51.000000
2,Sainato's at Rivergate,4.0,109,27.250000
3,OSTERIA di AL,4.0,87,21.750000
4,Il Venetian,4.5,91,20.222222


In [53]:
url='https://api.yelp.com/v3/businesses/matches'
params={"zip_code": "94109","country": "US"}
req = requests.get(url, headers=headers, params=params)
print('The status code is {}'.format(req.status_code))
print(req.text)

The status code is 400
{"error": {"code": "VALIDATION_ERROR", "description": "name is a required parameter.", "field": "name", "instance": null}}


In [10]:

# Define my parameters of the search
# BUSINESS SEARCH PARAMETERS - EXAMPLE
PARAMETERS = {'term': 'food','limit': 50,'zip_code': 44113}

# Make a request to the Yelp API
response = requests.get(url = ENDPOINT,
                        params = PARAMETERS,
                        headers = HEADERS)

# Conver the JSON String
business_data = response.json()

# print the response
print(json.dumps(business_data, indent = 3))

{
   "error": {
      "code": "VALIDATION_ERROR",
      "description": "Please specify a location or a latitude and longitude"
   }
}
